In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import BasicIterativeMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

2024-02-26 10:24:18.557827: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 10:24:18.560393: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 10:24:18.589042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 10:24:18.589094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 10:24:18.590047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

Ready to generate train and test datasets


In [4]:
model = joblib.load("/home/jovyan/MQTTset/dl/new_dl/dnn.joblib")

2024-02-26 10:24:25.224780: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://8e091f35b3f0428eb0808603c51ebd1c: INVALID_ARGUMENT: ram://8e091f35b3f0428eb0808603c51ebd1c is a directory.


In [5]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=6,
    input_shape=(33,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [6]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 640us/step
Accuracy: 0.06002618592003223
AUC: 0.6019832832475696

macro
Precision: 0.8389884895660815
Recall: 0.20656330041322138
F1 Score: 0.07969956012320527

weighted
Precision: 0.9681058073601138
Recall: 0.06002618592003223
F1 Score: 0.05179905604065773

Mean FNR: 0.7934366995867785
Mean TNR: 0.8379890013748281
Mean FPR: 0.16201099862517185
Mean TPR: 0.20656330041322138


In [8]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.05)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 594us/step
Accuracy: 0.2622519891227717
AUC: 0.624969225525222

macro
Precision: 0.4218070863256342
Recall: 0.2457362073214268
F1 Score: 0.15630167225455008

weighted
Precision: 0.732751780605736
Recall: 0.2622519891227717
F1 Score: 0.22384832822592032

Mean FNR: 0.7542637926785732
Mean TNR: 0.8467758517406093
Mean FPR: 0.1532241482593905
Mean TPR: 0.2457362073214268


In [6]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 586us/step
Accuracy: 0.3399738140799678
AUC: 0.5113734679859715

macro
Precision: 0.1405286380904408
Recall: 0.17724833856592973
F1 Score: 0.12740701834917934

weighted
Precision: 0.2154391893108761
Recall: 0.3399738140799678
F1 Score: 0.208135675075947

Mean FNR: 0.8227516614340703
Mean TNR: 0.8279783296272866
Mean FPR: 0.17202167037271343
Mean TPR: 0.17724833856592973


In [8]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.15)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 574us/step
Accuracy: 0.3220465303655957
AUC: 0.47507763485645377

macro
Precision: 0.1644071350360811
Recall: 0.15259951436548358
F1 Score: 0.1070994738850578

weighted
Precision: 0.22874126633445258
Recall: 0.3220465303655957
F1 Score: 0.20365767485657035

Mean FNR: 0.8474004856345164
Mean TNR: 0.8284018289143177
Mean FPR: 0.1715981710856823
Mean TPR: 0.15259951436548358


In [10]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.2)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [11]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 579us/step
Accuracy: 0.3284822237889012
AUC: 0.473288203957146

macro
Precision: 0.20148596906211
Recall: 0.14845306990480245
F1 Score: 0.09802112369297668

weighted
Precision: 0.3229207362988222
Recall: 0.3284822237889012
F1 Score: 0.20295855755491826

Mean FNR: 0.8515469300951976
Mean TNR: 0.8269899581054238
Mean FPR: 0.17301004189457628
Mean TPR: 0.14845306990480245


In [12]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.25)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [13]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 578us/step
Accuracy: 0.316265484943096
AUC: 0.4769162356733166

macro
Precision: 0.3167606515948424
Recall: 0.14382457679728364
F1 Score: 0.09852189077902503

weighted
Precision: 0.3970746690459797
Recall: 0.316265484943096
F1 Score: 0.20357676086666607

Mean FNR: 0.8561754232027164
Mean TNR: 0.8279531692206582
Mean FPR: 0.1720468307793418
Mean TPR: 0.14382457679728364


In [14]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.3)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [15]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 575us/step
Accuracy: 0.3170309195286534
AUC: 0.4716245410261512

macro
Precision: 0.1320374735812414
Recall: 0.14630590782772088
F1 Score: 0.09424074680238377

weighted
Precision: 0.33249163948374627
Recall: 0.3170309195286534
F1 Score: 0.2055580669913406

Mean FNR: 0.8536940921722791
Mean TNR: 0.829475213842667
Mean FPR: 0.17052478615733313
Mean TPR: 0.14630590782772088
